In [ ]:
TRAIN_DATASET_PATH_P = 'data/Processed_Data'
IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS = 240, 240, 1
NUM_CLASSES = 1

In [ ]:
model = unet()
model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss,
                  metrics=[dice_coef, 'binary_accuracy'])
model.summary()

In [ ]:
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

model.compile(optimizer=Adam(lr=2e-4), 
              loss=[dice_coef_loss], 
           metrics = [dice_coef, 'binary_accuracy'])


# create a data generator for training, validation, and testing data
data_gen = ImageDataGenerator(rescale=1./255)

train_data = data_gen.flow_from_directory(os.path.join(TRAIN_DATASET_PATH_P, 'train'),
                                          target_size=(IMG_HEIGHT, IMG_WIDTH),
                                          color_mode='grayscale',
                                          batch_size=32,
                                          class_mode='input')

valid_data = data_gen.flow_from_directory(os.path.join(TRAIN_DATASET_PATH_P, 'validation'),
                                          target_size=(IMG_HEIGHT, IMG_WIDTH),
                                          color_mode='grayscale',
                                          batch_size=32,
                                          class_mode='input')

test_data = data_gen.flow_from_directory(os.path.join(TRAIN_DATASET_PATH_P, 'test'),
                                          target_size=(IMG_HEIGHT, IMG_WIDTH),
                                          color_mode='grayscale',
                                          batch_size=1,
                                          class_mode=None)

# loss_history = model.fit(x = train_vol,
#                        y = train_seg,
#                          batch_size = 16,
#                   epochs = 50,
#                   validation_data =(test_vol,test_seg) ,
#                   callbacks=callbacks_list)

# train the model
history = model.fit(train_data, steps_per_epoch = 526 // 32,  epochs=3, validation_data=valid_data)

In [ ]:
import matplotlib.pyplot as plt

# load the test image, ground truth, and prediction
dir = os.path.join(TRAIN_DATASET_PATH_P, 'test', '010')


test_image = np.array(Image.open(os.path.join(dir,'flair.jpg')).convert('L').resize((IMG_HEIGHT, IMG_WIDTH)))
test_seg = np.array(Image.open(os.path.join(dir,'seg.jpg')).convert('L').resize((IMG_HEIGHT, IMG_WIDTH)))
pred_seg = model.predict(np.expand_dims(test_image, axis=0))[0,:,:,:]

# plot the test image, ground truth, and prediction
fig, axs = plt.subplots(1, 3, figsize=(10, 5))

axs[0].imshow(test_image, cmap='gray')
axs[0].set_title('Test Image')

axs[1].imshow(test_seg, cmap='gray')
axs[1].set_title('Ground Truth')

axs[2].imshow(pred_seg, cmap='gray')
axs[2].set_title('Prediction')

plt.show()